# Introduction

Favorita Stores, based in Ecuador, aims to forecast sales across thousands of product families.

The dataset includes information such as dates, store and product details, whether an item was on promotion, and historical sales figures. Additional supplementary data is also provided to help build a more accurate model.

While this is technically a time series problem, it is not a typical time series forecasting task. Since we need to predict sales for multiple products across multiple stores for the same and future dates, a regression-based approach is more appropriate than traditional time series models.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing all the datasets 👇**

In [ ]:
df_holiday = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
df_oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
df_sample_submission = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')
df_stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
df_test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
df_train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
df_transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

# Data pre-processing

In [ ]:
df_holiday.info()
# df_holiday.head()

In [ ]:
df_holiday['date_v2'] = pd.to_datetime(df_holiday['date'])
df_holiday['type'].value_counts()

In [ ]:
df_holiday[(df_holiday['transferred']== False) & (df_holiday['type']=='Holiday')]['date'].value_counts()

df_holiday[(df_holiday['date_v2']== pd.to_datetime('2017-06-25'))]
# 2017-06-25

df_holiday_f = df_holiday[(df_holiday['transferred']== False) & (df_holiday['type']=='Holiday')][['date_v2','locale','locale_name']]['date_v2'].value_counts().reset_index()
# df_holiday_f['date'].nunique()
df_holiday_f.rename(columns={'count':'holiday_count'},inplace=True)
df_holiday_f.tail()

In [ ]:
df_oil.info()
# df_sample_submission.head()
df_oil['date_v2'] = pd.to_datetime(df_oil['date'])
df_oil.rename(columns={'dcoilwtico':'oil_price'},inplace=True)
df_oil.loc[0,'oil_price'] = df_oil.iloc[1]['oil_price']
plt.plot(df_oil['date_v2'],df_oil['oil_price'])
df_oil['oil_price_v2'] = df_oil.oil_price.interpolate(method="linear")

In [ ]:
df_oil.isna().sum()

In [ ]:
df_oil_h = df_oil[['date_v2','oil_price_v2']]
df_oil_h

In [ ]:
df_stores.info()
df_stores.head()

In [ ]:
df_stores.isna().sum()
df_stores.nunique()

In [ ]:
df_test['date_v2'] = pd.to_datetime(df_test['date'])
df_test.head()
# df_test.info()

In [ ]:
# df_train.info()
# df_train.describe(include='all')
df_train['date_v2'] = pd.to_datetime(df_train['date'])
df_train.head()

# Merging the subests data with Train and Test dataset

In [ ]:
df_train_o = df_train.merge(df_oil_h,how='left',left_on='date_v2',right_on='date_v2')
df_train_o.head()

In [ ]:
df_test_o = df_test.merge(df_oil_h,how='left',left_on='date_v2',right_on='date_v2')
df_test_o.head()

In [ ]:
# df_train_o['oil_price_v2'].fillna(df_train_o['oil_price_v2'].mean(),inplace=True)
df_train_o.fillna({'oil_price_v2': df_train_o['oil_price_v2'].mean()}, inplace=True)
df_test_o.fillna({'oil_price_v2': df_test_o['oil_price_v2'].mean()}, inplace=True)

**Note: Tried to create a function for populating Holiday count but due to large number of records it was taking a lot of time in executing so did not use it further**

In [ ]:
def f_get_holiday(dt):
    # holiday_count = 0
    holiday_count_s = df_holiday_f[df_holiday_f['date_v2']==dt]['count']
    if holiday_count_s.shape[0] > 0:
        return holiday_count_s.values[0]
    else:
        return 0
    # print(holiday_count.shape,holiday_count)
# f_get_holiday(pd.to_datetime('2017-12-26'))
# df_train['Holiday'] = df_train['date_v2'].apply(f_get_holiday)
# df_test['Holiday'] = df_test['date_v2'].apply(f_get_holiday)

In [ ]:
df_train_h = df_train_o.merge(df_holiday_f,how = 'left',right_on = 'date_v2',left_on='date_v2',suffixes=('','_hol'))
df_train_h.drop('date',axis=1,inplace=True)
df_train_h.fillna({'holiday_count':0},inplace=True)
df_train_h['holiday_count'] = df_train_h['holiday_count'].astype('int32')
df_train_h.head()

In [ ]:
df_test_h = df_test_o.merge(df_holiday_f,how = 'left',right_on = 'date_v2',left_on='date_v2',suffixes=('','_hol'))
df_test_h.drop('date',axis=1,inplace=True)
df_test_h.fillna({'holiday_count': 0},inplace=True)
df_test_h['holiday_count'] = df_test_h['holiday_count'].astype('int32')
df_test_h.head()

In [ ]:
df_train_h['dt_month_day'] = df_train_h['date_v2'].dt.day
df_train_h['dt_month'] = df_train_h['date_v2'].dt.month
df_train_h['dt_week_day'] = df_train_h['date_v2'].dt.dayofweek

In [ ]:
df_test_h['dt_month_day'] = df_test_h['date_v2'].dt.day
df_test_h['dt_month'] = df_test_h['date_v2'].dt.month
df_test_h['dt_week_day'] = df_test_h['date_v2'].dt.dayofweek

In [ ]:
df_train_h.head()

In [ ]:
df_test_h.head()

In [ ]:
df_train_s = df_train_h.merge(df_stores,how='left',left_on='store_nbr',right_on='store_nbr',suffixes=('','_str'))
df_test_s = df_test_h.merge(df_stores,how='left',left_on='store_nbr',right_on='store_nbr',suffixes=('','_str'))

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(x='date_v2',y='sales',data = df_train_s)
plt.show()

In [ ]:
sns.histplot(df_train_s['sales'], bins=100, kde=True)
plt.show()

**⭐ Final Train and Test datasets**

In [ ]:
df_train_s[['store_nbr', 'family', 'sales', 'onpromotion',
       'oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month',
       'dt_week_day', 'city', 'state', 'type', 'cluster']]

In [ ]:
df_test_s[['store_nbr', 'family', 'onpromotion',
       'oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month',
       'dt_week_day', 'city', 'state', 'type', 'cluster']]

In [ ]:
df_train_s.isna().sum()

In [ ]:
df_test_s.isna().sum()

# Splitting X, Y and applying Encoding and Scaling

In [ ]:
df_train_y = df_train_s['sales'].copy()
df_train_X = df_train_s[['store_nbr', 'family', 'onpromotion',
       'oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month',
       'dt_week_day', 'city', 'state', 'type', 'cluster']].copy()
df_test_X = df_test_s[['store_nbr', 'family', 'onpromotion',
       'oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month',
       'dt_week_day', 'city', 'state', 'type', 'cluster']].copy()

In [ ]:
print(df_train_y.shape,df_train_X.shape)

**Trying K-fold**

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from category_encoders import TargetEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import StandardScaler #Standardization
from xgboost import XGBRegressor

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kf.split(df_train_X):
    X_train, X_test = df_train_X.iloc[train_index].copy(), df_train_X.iloc[test_index].copy()
    y_train, y_test = df_train_y.iloc[train_index].copy(), df_train_y.iloc[test_index].copy()
    
    # print(X_train.shape,X_test.shape)
    # print(y_train.shape,y_test.shape)

    target_encoder_model = TargetEncoder(cols=['family','city','state','type'])
    X_train[['family','city','state','type']] = target_encoder_model.fit_transform(X_train[['family','city','state','type']], y_train)
    X_test[['family','city','state','type']] = target_encoder_model.transform(X_test[['family','city','state','type']])

    std_scaler = StandardScaler()

    X_train[['store_nbr', 'onpromotion','oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month','dt_week_day', 'cluster']] = std_scaler.fit_transform(X_train[['store_nbr', 'onpromotion','oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month','dt_week_day', 'cluster']]) #transform and fit on train
    X_test[['store_nbr', 'onpromotion','oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month','dt_week_day', 'cluster']] = std_scaler.transform(X_test[['store_nbr', 'onpromotion','oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month','dt_week_day', 'cluster']])
    
    X_train.drop(columns='state',axis=1,inplace=True)
    X_test.drop(columns='state',axis=1,inplace=True)
    
    log_mean_target = np.log1p(y_train.mean())
    
    xgb_model = XGBRegressor(
        n_estimators=100,
        objective='count:poisson',
        base_score = log_mean_target
    )

    xgb_model.fit(X_train, y_train)
    r2 = xgb_model.score(X_test, y_test)
    

    y_pred = xgb_model.predict(X_test)
    rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))

    print("R² scores:", r2)
    print("RMSLE scores:", rmsle)

    

In [ ]:
from category_encoders import TargetEncoder
target_encoder_model = TargetEncoder(cols=['family','city','state','type'])
df_train_X[['family','city','state','type']] = target_encoder_model.fit_transform(df_train_X[['family','city','state','type']], df_train_y)

df_test_X[['family','city','state','type']] = target_encoder_model.transform(df_test_X[['family', 'city', 'state', 'type']])

In [ ]:
from sklearn.preprocessing import StandardScaler #Standardization
std_scaler = StandardScaler()

df_train_X[['store_nbr', 'onpromotion','oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month','dt_week_day', 'cluster']] = std_scaler.fit_transform(df_train_X[['store_nbr', 'onpromotion','oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month','dt_week_day', 'cluster']]) #transform and fit on train

In [ ]:
df_test_std = std_scaler.transform(df_test_X[['store_nbr', 'onpromotion','oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month','dt_week_day', 'cluster']])

# df_test_X[['store_nbr', 'onpromotion','oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month','dt_week_day', 'cluster']] = df_test_X[['store_nbr', 'onpromotion','oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month','dt_week_day', 'cluster']].astype('float64').copy()
df_test_X[['store_nbr', 'onpromotion','oil_price_v2', 'holiday_count', 'dt_month_day', 'dt_month','dt_week_day', 'cluster']] = df_test_std

In [ ]:
X_train.columns

**Checking VIF for Multicollinearity**

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
X_t = df_train_X[['store_nbr', 'family', 'onpromotion', 'oil_price_v2', 'holiday_count',
       'dt_month_day', 'dt_month', 'dt_week_day','state' ,'city', 'type',
       'cluster']].copy()
vif['Features'] = X_t.columns
vif['VIF'] = [variance_inflation_factor(X_t.values, i) for i in range(X_t.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

**Removing State column as State and City are causing Multicollinearity**

In [ ]:
df_train_X.drop(columns='state',axis=1,inplace=True)
df_test_X.drop(columns='state',axis=1,inplace=True)

# Creating Models

**XGBoost for Regression**

In [ ]:
print(df_train_y[df_train_y > 0].count())
print(df_train_y[df_train_y == 0].count())
print(df_train_y.shape)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

log_mean_target = np.log1p(df_train_y.mean())
print(log_mean_target)

xgb_model = XGBRegressor(
    n_estimators=100,
    objective='count:poisson',
    base_score = log_mean_target
)
# Fit the model
xgb_model.fit(df_train_X, df_train_y)

**Test data prediction for final submission**

In [ ]:
y_pred_test = xgb_model.predict(df_test_X)

In [ ]:
y_pred_test

In [ ]:
df_test_predicted = pd.DataFrame({'id':df_test['id'].values,'sales':y_pred_test})
df_test_predicted.to_csv('submission.csv',index=False)